<h1>GenePlanet Homework - Task 2</h1>

<h3>Calculate the individual’s Polygenic Risk Score (PRS) for Breast Cancer based on the 77 SNPs as explained in the article*.</h3>

<i>*Prediction of Breast Cancer Risk Based on Profiling With Common Genetic Variants
<br>(J Natl Cancer Inst. 2015 Apr 8;107(5). pii: djv036. doi: 10.1093/jnci/djv036. Print 2015 May.)</i>


<h2>Background</h2>

Breast cancer is the most common cancer among Western women. However not all women carry the same risk of developing breast cancer. Therefore the stratification of women based on the risk for developing breast cancer, would improve the effectiveness of the current strategies for detection and treatment of breast cancer as it would allow to tailor these strategies based on the individual’s own risk of developing breast cancer.

While high-risk mutations (such as mutations in genes BRCA1 and BRCA2) can explain a small proportion of breast cancer cases, Genome Wide Association Studies (GWAS) have led to discovery that multiple common low-risk variants (single nucleotide polymorphisms - SNPs) can have a combined effect of increasing the individual's risk of developing breast cancer.

To get a better understanding of how the combined effects of low risk variants influence the risk of expression of a specific phenotype (in our case cancer), we calculate Polygenic Risk Score (PRS). PRS is a single value estimate of the individual's propensity to a speficit phenotype. PRS is calculated as a sum of all individual SNP contributions for the specific phenotype. Each contribution is the effect size (log(odds ratio)) multiplied by the number of matching allels (0,1 or 2) in the SNP, based on which the odds ratio (OR) was caluclated. OR is a ratio of odds for expressing phenotype due to the individual SNP (nominator) and odds of not expressing that phenotype due to the same SNP (denominator).

The authors in the article* identified 77 SNPs which contribute to the risk of developing breast cancer. They observed that individuals with the top 1 % of PRS are 3 times as likely to develop breast cancer as individuals with the PRS value in the middle quantile.<br>The information required to calculate the PRS is presented in the Table 4 (Supplmentary Material) and includes all SNPs and ORs as well as allele information for each SNPs. The ORs were calculated based on the minor allele. <b>Based on this information we can calculate the PRS for any individual as long as we have the individuals 77 SNPs.</b>


<h2>Getting the individual's SNPs</h2>
    
Ensembl website has a collection of anonymous genomes of individuals. We are going to download the SNPs of the anonymous female from Great Britain (sample name: <b><a href="http://www.internationalgenome.org/data-portal/sample/HG00099">HG00099</a></b>), who was one of the participants in the 1000G project. We can obtain the sample name by browsing the Ensembl website.

There are two ways to obtain the SNPs for this sample. 
1. From Ensembl website we can download many VCF files that combined contain all the 77 SNPs. We then filter each file based on the specific sample name.

2. We can communicate with Ensembl database through their API. We create a script that can query the database for each SNP. We receive all the samples for specific SNP in JSON format which we then parse for specific sample name.

While both options have pros and cons we chose the option 2 (the script is located in the <b>ensembl_util.py</b>).<br> 
This option is internet dependent as it does not store information locally. However as long as we have access to the database we can retrieve the latest Ensembl information at any time and we do not need to worry about storing/losing the information locally.

<i>Right now the script returns only SNPs for the individual sample. However we can expand the script to retrive any information that is stored in the Ensembl database.</i>


<h2>PRS Calculation</h2>

In the article* the PRS for the individuals is defined as:

“
<b>PRS = b1*x1 + b2*x2 + .... bk*xk …. + bn*xn

Where bk is the per-allele log odds ratio (OR) for breast cancer associated with the minor allele for SNP k, and xk the number of alleles for the same SNP (0, 1 or 2), and n=77 is the total number of SNPs.</b>"


<h2>Implementing the PRS calculation in code</h2>

In [31]:
# importing libraries
import pandas as pd
import ensembl_utils # our custom script to communicate with Ensembl database
import numpy as np

<h3> Loading Table 4 </h3>

In [32]:
# loading the Table 4 information to memory
df = pd.read_csv("./data/77SNP_Breast_Cancer_Table4.csv")
df.set_index("SNP", inplace=True)

# table summary
print(df.info())

# showing first 5 rows of the Table
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 77 entries, rs78540526 to rs11075995
Data columns (total 6 columns):
Alleles                77 non-null object
Locus                  77 non-null object
Chromosome             77 non-null int64
All breast cancers     77 non-null float64
ER-positive disease    77 non-null float64
ER-negative disease    77 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 4.2+ KB
None


,Alleles,Locus,Chromosome,All breast cancers,ER-positive disease,ER-negative disease
SNP,,,,,,
rs78540526,C/T,CCND1,11,1.1761,1.2342,0.9049
rs75915166,C/A,CCND1,11,1.0239,1.0189,1.1326
rs554219,C/G,CCND1,11,1.1238,1.1402,1.0094
rs7726159,C/A,TERT,5,1.0359,1.0442,1.0306
rs10069690,C/T,TERT,5,1.0242,0.9948,1.1200


<h3>Getting the SNPs of the sample</h3>

In [33]:
# Getting the 77 SNPs of anonymous female individual
# from Ensembl database (Individual HG00099 (Female) EUR, GBR), found
# by browsing the Ensemble database.
# http://www.internationalgenome.org/data-portal/sample/HG00099


# we call our custom function that queries the Ensembl database
# and returns the SNPs. We supply function with the list of SNPs and sample name.
# (df.index is the list of all SNPS from the above table)
ind_snps = ensembl_utils.ensembl_snp_collector(df.index, "HG00099") 

[INFO] Collecting 77 SNPs for sample HG00099
[INFO] Collecting (1/77): rs78540526
[INFO] Collecting (2/77): rs75915166
[INFO] Collecting (3/77): rs554219
[INFO] No indivudal SNP. Getting reference allele pair.
[INFO] Collecting (4/77): rs7726159
[INFO] Collecting (5/77): rs10069690
[INFO] Collecting (6/77): rs2736108
[INFO] Collecting (7/77): rs2588809
[INFO] Collecting (8/77): rs999737
[INFO] Collecting (9/77): rs10759243
[INFO] Collecting (10/77): rs865686
[INFO] Collecting (11/77): rs2981579
[INFO] Collecting (12/77): rs11199914
[INFO] Collecting (13/77): rs7072776
[INFO] Collecting (14/77): rs11814448
[INFO] Collecting (15/77): rs13387042
[INFO] Collecting (16/77): rs16857609
[INFO] Collecting (17/77): rs11552449
[INFO] Collecting (18/77): rs11249433
[INFO] Collecting (19/77): rs1045485
[INFO] Collecting (20/77): rs4973768
[INFO] Collecting (21/77): rs10941679
[INFO] Collecting (22/77): rs889312
[INFO] Collecting (23/77): rs12662670
[INFO] Collecting (24/77): rs2046210
[INFO] Colle

<h3>Calculate PRS for the sample</h3>

In [34]:
# Append Sample's Allelles column to the Table
df['SAMPLE'] = pd.DataFrame([ind_snps["snps"]]).T

# showing first 5 rows of the Table with indcluded SAMPLE allells column
df.head(5)

,Alleles,Locus,Chromosome,All breast cancers,ER-positive disease,ER-negative disease,SAMPLE
SNP,,,,,,,
rs78540526,C/T,CCND1,11,1.1761,1.2342,0.9049,C|C
rs75915166,C/A,CCND1,11,1.0239,1.0189,1.1326,C|C
rs554219,C/G,CCND1,11,1.1238,1.1402,1.0094,C|
rs7726159,C/A,TERT,5,1.0359,1.0442,1.0306,C|C
rs10069690,C/T,TERT,5,1.0242,0.9948,1.1200,C|C


In [36]:
# PRS = B1*x1 + B2*x2 + ... Bk*xk ... +Bn*Xn
# Bk - per-allele log odds ratio (OR) associated with minor allele for SNP k
# xk - the number of alleles for the same SNP (0, 1 or 2)
# n - number of SNPs (len(df.index) = 77)

# Get the X_k (either 0,1 or 2)
X_k = {}
for row in df.index:
    # get minor allele
    ref = df.loc[row].Alleles.split("/")[-1]
    sample = df.loc[row].SAMPLE.split("|")
    # count occurances of the minor allele in the sample
    X_k[row] = sample.count(ref)

# Append X_k to the Table 4
df["X_k"] = pd.DataFrame([X_k]).T

# Show the last 5 rows of the Table 4 with appended X_k column
df.tail()

,Alleles,Locus,Chromosome,All breast cancers,ER-positive disease,ER-negative disease,SAMPLE,X_k
SNP,,,,,,,,
rs6001930,T/C,MKL1,22,1.1345,1.1313,1.1214,T|T,0
rs4245739,A/C,MDM4,1,1.0291,0.9961,1.1640,C|A,1
rs6678914,G/A,LGR6,1,0.9890,1.0058,0.9146,G|G,0
rs12710696,G/A,2p24.1,2,1.0387,1.0128,1.1054,C|T,0
rs11075995,A/T,FTO,16,1.0368,1.0204,1.1016,T|T,2


In [37]:
# Caluclate PRS for each type of Breast cancer

# Initiating the values
PRS_all = 0
PRS_ER_neg = 0
PRS_ER_pos = 0

# Iterating over each row in the table and summing the contribution of each SNP
# to obtain the final PRS (based on the PRS equation).
# (np.log() is base e logarithm function)

for row in df.index:
    PRS_all += np.log(df.loc[row]["All breast cancers"])*df.loc[row]["X_k"]
    PRS_ER_pos += np.log(df.loc[row]["ER-positive disease"])*df.loc[row]["X_k"]
    PRS_ER_neg += np.log(df.loc[row]["ER-negative disease"])*df.loc[row]["X_k"]

print("All breast cancers PRS value: {}".format(PRS_all))
print("ER-positive disease PRS value: {}".format(PRS_ER_pos))
print("ER-negative disease PRS value: {}".format(PRS_ER_neg))

All breast cancers PRS value: 0.761568621021309
ER-positive disease PRS value: 0.7483809608560952
ER-negative disease PRS value: 0.8013281458141586


<b>PRS for the sample calculated.<b>

In [38]:
# showing selected columns of the Table for reference
# we can see that for rs554219 the minor allelle information was missing from the information
# returned by our query.

print(df[["Alleles","SAMPLE", "X_k"]])

           Alleles SAMPLE  X_k
SNP                           
rs78540526     C/T    C|C    0
rs75915166     C/A    C|C    0
rs554219       C/G     C|    0
rs7726159      C/A    C|C    0
rs10069690     C/T    C|C    0
rs2736108      C/T    C|T    1
rs2588809      C/T    C|C    0
rs999737       C/T    C|C    0
rs10759243     C/A    C|C    0
rs865686       T/G    T|G    1
rs2981579      G/A    G|A    1
rs11199914     C/T    C|T    1
rs7072776      G/A    G|G    0
rs11814448     A/C    A|A    0
rs13387042     A/G    A|A    0
rs16857609     C/T    T|C    1
rs11552449     C/T    C|C    0
rs11249433     A/G    G|A    1
rs1045485      G/C    G|G    0
rs4973768      C/T    C|C    0
rs10941679     A/G    G|G    2
rs889312       A/C    A|C    1
rs12662670     T/G    T|G    1
rs2046210      G/A    G|A    1
rs13281615     A/G    G|G    2
rs1011970      G/T    G|T    1
rs2380205      C/T    C|C    0
rs10995190     G/A    A|G    1
rs704010       C/T    C|C    0
rs3817198      T/C    T|T    0
...     